# Задача № 2 (Семинар 11)

In [3]:
import pandas as pd 
import numpy as np
from numpy.linalg import inv
from sklearn.metrics import mean_squared_error
import statsmodels.api as sm

In [3]:
df = pd.read_excel('sem_11.xlsx')

In [4]:
df.head()

,CONST,Y1,X1,X2,U1,U2
0,1,56.7,104.3,270,97.8,102.0054
1,1,64.5,94.2,172,105.7,99.5694
2,1,53.3,102.8,324,103.3,106.1924
3,1,82.6,98.7,428,95.1,93.8637
4,1,62.0,99.8,420,100.5,100.2990


In [5]:
y_matrix = df.drop(['X1', 'X2', 'U1', 'U2', 'CONST'], axis = 1)
x_matrix = df.drop(['Y1', 'U1', 'U2'], axis = 1)
z_matrix = df.drop(['Y1', 'X2', 'U1'], axis = 1)

In [37]:
x_matrix #матрица регрессоров

,CONST,X1,X2
0,1,104.3,270
1,1,94.2,172
2,1,102.8,324
3,1,98.7,428
4,1,99.8,420
5,1,100.5,473
6,1,112.8,82
7,1,106.7,276
8,1,100.3,409
9,1,105.0,191


In [38]:
y_matrix # зависимая переменная

,Y1
0,56.7
1,64.5
2,53.3
3,82.6
4,62.0
5,61.3
6,25.7
7,36.1
8,69.7
9,46.4


In [39]:
z_matrix # матрица условий

,CONST,X1,U2
0,1,104.3,102.0054
1,1,94.2,99.5694
2,1,102.8,106.1924
3,1,98.7,93.8637
4,1,99.8,100.2990
5,1,100.5,101.9070
6,1,112.8,124.1928
7,1,106.7,107.4469
8,1,100.3,102.3060
9,1,105.0,106.9950


In [40]:
# транспонируем матрицы для дальнейших вычислений
x_tr = x_matrix.transpose()
z_tr = z_matrix.transpose()

In [10]:
ztrx = np.dot(z_tr,x_matrix)

In [11]:
inverted = inv(ztrx)

In [12]:
ztry = np.dot(z_tr, y_matrix)

In [13]:
betta_iv = np.dot(inverted,ztry)

In [14]:
betta_iv

array([[ 1.87503732e+02],
       [-1.44402792e+00],
       [ 6.14489830e-02]])

In [41]:
y_hat = np.dot(z_matrix,betta_iv) # нахожу оценку y с крышей
y_hat

array([[43.1597486 ],
       [57.59474083],
       [45.58307737],
       [50.74600575],
       [49.55301768],
       [48.6410081 ],
       [32.24890448],
       [40.02845624],
       [48.95433183],
       [42.45553491],
       [41.92518265],
       [41.17634238],
       [35.62205772],
       [41.27733912],
       [41.23988364]])

In [42]:
var_py = mean_squared_error(y_matrix, y_hat) #не скорректированная дисперсия
var_y = (var_py*len(z_matrix))/(len(z_matrix)-3) # скорректированная, искомая дисперсия
print('sigma = ', var_y)

sigma =  220.77681412636255


In [43]:
#найдем матрицу ковариации
xtz = np.dot(x_tr, z_matrix)
ztz = np.dot(z_tr, z_matrix)
invz = inv(ztz)
xtzinvz = np.dot(xtz,invz)
covar = np.dot(xtzinvz,ztrx)
covar = var_y * covar
print(covar)

[[3.31165221e+03 3.44168976e+05 8.71406085e+05]
 [3.44168976e+05 3.58365168e+07 8.94408159e+07]
 [8.71406085e+05 8.94408159e+07 2.57692375e+08]]


### Проверим готовой функцией результаты (как в пакете)

In [44]:
endog_iv = x_matrix.drop('CONST', axis = 1)
instruments_iv = df.drop(['Y1', 'CONST', 'X1', 'X2'], axis = 1)
from linearmodels.iv import IV2SLS
iv = IV2SLS(dependent=y_matrix,
            exog=df['CONST'],
            endog=endog_iv,
            instruments=instruments_iv).fit(cov_type='unadjusted')

print(iv.summary)

                          IV-2SLS Estimation Summary                          
Dep. Variable:                     Y1   R-squared:                      0.7144
Estimator:                    IV-2SLS   Adj. R-squared:                 0.6668
No. Observations:                  15   F-statistic:                    38.677
Date:                Sun, Mar 31 2019   P-value (F-stat)                0.0000
Time:                        14:04:16   Distribution:                  chi2(2)
Cov. Estimator:            unadjusted                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
CONST          180.94     77.028     2.3491     0.0188      29.972      331.92
X1            -1.3872     0.6733    -2.0602     0.03

### Оценка коэффициентов методом 2-мнк по формуле, руками

In [46]:
# найдем 2-мнк оценку бетта, для этого введем новую матрицу Z, которая состит из инструментальных переменных U1, U2
z = df.drop(['Y1', 'X2'], axis = 1)
z

,CONST,X1,U1,U2
0,1,104.3,97.8,102.0054
1,1,94.2,105.7,99.5694
2,1,102.8,103.3,106.1924
3,1,98.7,95.1,93.8637
4,1,99.8,100.5,100.2990
5,1,100.5,101.4,101.9070
6,1,112.8,110.1,124.1928
7,1,106.7,100.7,107.4469
8,1,100.3,102.0,102.3060
9,1,105.0,101.9,106.9950


In [47]:
# находим оценку коэффициентов методом 2-МНК
zt = z.transpose()
ztz = np.dot(zt,z)
inv_z = inv(ztz)
x_t_z = np.dot(x_tr, z)
z_t_x = np.dot(zt, x_matrix)
fpart = np.dot(x_t_z, inv_z)
first = np.dot(fpart, z_t_x)
second = inv(first)
third = np.dot(second, x_t_z)
fourth = np.dot(third,inv_z)
z_t_y = np.dot(zt, y_matrix)
betta_2sls = np.dot(fourth,z_t_y)

In [21]:
print(betta_2sls)

[[ 1.59133541e+02]
 [-1.20745951e+00]
 [ 7.58311423e-02]]


In [22]:
print("%.3f" % (betta_2sls[0]))

159.134


In [23]:
print("%.3f" % (betta_2sls[1]))

-1.207


In [24]:
print("%.3f" % (betta_2sls[2]))

0.076


In [25]:
x_matrix.shape

(15, 3)

In [26]:
z.shape

(15, 4)

In [27]:
y_matrix.shape

(15, 1)

### Алгоритмический метод

In [28]:
# 1 шаг: определение матрицы Nz
N = np.dot(z,inv_z)
Nz = np.dot(N,zt)
print(Nz)

[[ 2.51982818e-01 -1.26128538e-01  5.69956929e-02  1.19133308e-01
   5.43195472e-02  5.31257438e-02 -8.76670160e-02  2.25361755e-01
   3.58056078e-02  1.44106105e-01  5.10269877e-03 -2.91582941e-02
  -5.29144533e-02  1.30586345e-01  2.19348680e-01]
 [-1.26128538e-01  6.72525359e-01  1.49022235e-01 -8.96627539e-02
   1.44615244e-01  1.60656603e-01 -1.62513575e-01 -1.06299391e-01
   1.96913719e-01  1.68772730e-03  1.38839661e-01  1.48633543e-01
  -3.82830137e-02 -3.78440075e-04 -8.96283782e-02]
 [ 5.69956929e-02  1.49022235e-01  8.34253457e-02 -3.72798070e-04
   6.32826413e-02  7.22577484e-02  6.42523904e-03  7.06432123e-02
   7.79486849e-02  7.30795156e-02  8.32597651e-02  8.08779436e-02
   3.99876439e-02  7.42474182e-02  6.89197124e-02]
 [ 1.19133308e-01 -8.96627539e-02 -3.72798069e-04  7.41349561e-01
   2.61793590e-01  1.71529499e-01  1.03956478e-01 -7.41833034e-02
   1.40967996e-01 -2.07872210e-02 -1.16776070e-01 -1.26051621e-01
  -1.94623735e-02 -6.64666789e-02 -2.49676123e-02]
 [ 5

In [29]:
x_hat = np.dot(Nz,x_matrix) #получили предсказанные значения регрессора 

In [30]:
print(x_hat)

[[  1.         104.30000001 327.48653   ]
 [  1.          94.2        293.52833512]
 [  1.         102.8        259.30480575]
 [  1.          98.7        502.0227223 ]
 [  1.          99.80000001 357.55279536]
 [  1.         100.5        327.37834705]
 [  1.         112.8        109.41137616]
 [  1.         106.7        243.79835037]
 [  1.         100.3        317.59864737]
 [  1.         105.         253.01336198]
 [  1.         105.6        180.34151106]
 [  1.         106.2        159.89131543]
 [  1.         110.3        124.06722616]
 [  1.         105.90000001 226.70599322]
 [  1.         105.8        264.89868275]]


In [31]:
# 2 шаг: оценка коэффициентов регрессии (y на предсказанное на предыдущем шаге х)
x_hat_t = x_hat.transpose()
first = np.dot(x_hat_t, x_matrix)
second = inv(first)
third = np.dot(second,x_hat_t)
b2sls = np.dot(third, y_matrix)

In [32]:
print(b2sls)

[[ 1.59133543e+02]
 [-1.20745953e+00]
 [ 7.58311416e-02]]


### Проверка полученных результатов

In [33]:
endog = x_matrix.drop(['X1','CONST'], axis = 1)
endog

,X2
0,270
1,172
2,324
3,428
4,420
5,473
6,82
7,276
8,409
9,191


In [34]:
exog_par = df.drop(['Y1', 'U1', 'U2', 'X2'], axis = 1)
exog_par

,CONST,X1
0,1,104.3
1,1,94.2
2,1,102.8
3,1,98.7
4,1,99.8
5,1,100.5
6,1,112.8
7,1,106.7
8,1,100.3
9,1,105.0


In [35]:
instruments = df.drop(['Y1', 'CONST', 'X2', 'X1'], axis = 1)
instruments

,U1,U2
0,97.8,102.0054
1,105.7,99.5694
2,103.3,106.1924
3,95.1,93.8637
4,100.5,100.2990
5,101.4,101.9070
6,110.1,124.1928
7,100.7,107.4469
8,102.0,102.3060
9,101.9,106.9950


In [36]:
from linearmodels.iv import IV2SLS
iv = IV2SLS(dependent=y_matrix,
            exog=exog_par,
            endog=endog,
            instruments=instruments).fit(cov_type='unadjusted')

print(iv.summary)

                          IV-2SLS Estimation Summary                          
Dep. Variable:                     Y1   R-squared:                      0.6778
Estimator:                    IV-2SLS   Adj. R-squared:                 0.6241
No. Observations:                  15   F-statistic:                    37.288
Date:                Sun, Mar 31 2019   P-value (F-stat)                0.0000
Time:                        14:02:59   Distribution:                  chi2(2)
Cov. Estimator:            unadjusted                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
CONST          159.13     77.910     2.0425     0.0411      6.4323      311.83
X1            -1.2075     0.6849    -1.7629     0.07

#### Видим, что оценки, найденные по формуле, затем алгоритмически в два шага совпадают с оценками, которые были найдены при помощи встроенной функции. Она абсолютно дублирует листинг, который выдает EViews.